<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#ComplementNB" data-toc-modified-id="ComplementNB-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>ComplementNB</a></span></li><li><span><a href="#RandomForestClassifier" data-toc-modified-id="RandomForestClassifier-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>RandomForestClassifier</a></span></li><li><span><a href="#SGDClassifier" data-toc-modified-id="SGDClassifier-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>SGDClassifier</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
#pip install pandarallel

In [2]:
import pandas as pd
import numpy as np

from time import time

from tqdm.notebook import tqdm
#tqdm.pandas()

import os

import re
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.naive_bayes import ComplementNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import cross_val_score, RandomizedSearchCV

from sklearn.metrics import f1_score, make_scorer

RANDOM_STATE = 4242

from sklearn.pipeline import Pipeline

In [19]:
#pip install spacy

In [21]:
import spacy

In [28]:
#!python -m spacy download en_core_web_sm

In [27]:
nlp = spacy.load('en_core_web_sm', disable = ['parser','ner'])

In [29]:
pth1 = 'datasets/toxic_comments.csv'
pth2 = '/datasets/toxic_comments.csv'

if os.path.exists(pth1):
    df = pd.read_csv(pth1, index_col=[0])
elif os.path.exists(pth2):
    df = pd.read_csv(pth2, index_col=[0])
else:
    print('Paths not found')

In [30]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [32]:
df['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

In [33]:
df['text'].duplicated().sum()

0

In [34]:
df.isna().sum()

text     0
toxic    0
dtype: int64

In [35]:
df['toxic'] = df['toxic'].astype('int8')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int8  
dtypes: int8(1), object(1)
memory usage: 2.6+ MB


- Посмотрели данные, из обработки только уменьшили размер датасета на 1 Мб, возможно это ускорит обучение потом.
- Есть дисбаланс классов - только 11% позитивных значений - возможно надо будет сделать балансировку.

In [36]:
def clear_text(text):
    text2 = re.sub(r'[^a-zA-Z ]', ' ', text)
    return " ".join(text2.split())

In [37]:
def lemmatizer(text):        
    sent = []
    doc = nlp(text)
    for word in doc:
        sent.append(word.lemma_)
    return " ".join(sent)

Сформируем обучающую и тестовую выборки.

In [38]:
target = df['toxic']

In [39]:
tqdm.pandas()

df['lemm_text'] = df['text'].progress_apply(lemmatizer)

  0%|          | 0/159292 [00:00<?, ?it/s]

In [40]:
clear_lemm_text = df['lemm_text'].apply(clear_text)

In [41]:
clear_lemm_text = clear_lemm_text.str.lower()
#corpus

In [42]:
df['lemm_text'] = pd.DataFrame(clear_lemm_text, index=df.index)

In [43]:
features_train, features_test, target_train, target_test = train_test_split(
    df['lemm_text'], 
    target, 
    test_size=0.1, 
    random_state=12345)

print(features_train.shape, target_train.shape, features_test.shape, target_test.shape)

(143362,) (143362,) (15930,) (15930,)


Загрузим стоп-слова.

In [44]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [66]:
count_tf_idf = TfidfVectorizer(stop_words=list(stopwords), lowercase=True)
count_tf_idf

TfidfVectorizer(stop_words=['which', 'hasn', 'after', 'yourselves', "mustn't",
                            'then', "needn't", "shouldn't", 'me', 'haven', 'by',
                            "hadn't", 't', 'mustn', 'my', 'ours', 'other',
                            "couldn't", 'will', "hasn't", 'hadn', "shan't",
                            'nor', 'he', 'll', 'isn', 's', 'wouldn', 'these',
                            'herself', ...])

Посчитаем TF-IDF для выборок. 

In [67]:
tf_idf = count_tf_idf.fit_transform(features_train)
tf_idf_te = count_tf_idf.transform(features_test)

In [47]:
tf_idf

<143362x147784 sparse matrix of type '<class 'numpy.float64'>'
	with 3763860 stored elements in Compressed Sparse Row format>

In [70]:
tf_idf_t_l = tf_idf_te.astype(str)
tf_idf_t_l

<15930x147784 sparse matrix of type '<class 'numpy.str_'>'
	with 405749 stored elements in Compressed Sparse Row format>

## Обучение

In [48]:
#Создам таблицу с F1 моделей
model = ['LogisticRegression', 'ComplementNB', 'RandomForestClassifier', 'SGDClassifier']    
model_f1 = pd.DataFrame(columns=['F1'], index=model)

Возьмем несколько моделей классификации sklearn для классификации текстовых документов.

### LogisticRegression

In [49]:
pipeline_lr = Pipeline(
    [
        ("vect", TfidfVectorizer(stop_words=list(stopwords), lowercase=True)),
        ("clf", LogisticRegression()),
    ]
)

In [50]:
#pipeline_lr.get_params().keys()

In [51]:
params_lr ={
    'clf__max_iter' : [300],
    'clf__warm_start' : [False],
    'clf__solver' : ['lbfgs'], 
    'clf__C' : [10] 
}  #np.arange(0, 1, 0.01), #'newton-cg', 'liblinear'

In [52]:
t0 = time()

model_lr = RandomizedSearchCV(estimator=pipeline_lr,
                             param_distributions = params_lr, n_iter = 7, cv=3, scoring = 'f1', 
                             n_jobs = -1, verbose = 2, random_state = RANDOM_STATE
                             )

#model_lr.fit(tf_idf, target_train)
model_lr.fit(features_train, target_train)

#model_lr.best_estimator_.transform(features_train)
#model_lr.transform(features_test)

print(f"LogisticRegression выполнена за {time() - t0:.3f}s")

print('Средняя оценка F1 модели LogisticRegression: %0.2f' % model_lr.best_score_)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 1 is smaller than n_iter=7. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 3 folds for each of 1 candidates, totalling 3 fits
LogisticRegression выполнена за 100.082s
Средняя оценка F1 модели LogisticRegression: 0.77


In [53]:
print('Лучшие параметры: {}\n'.format(model_lr.best_params_))
model_f1['F1']['LogisticRegression'] = round(model_lr.best_score_,4)

Лучшие параметры: {'clf__warm_start': False, 'clf__solver': 'lbfgs', 'clf__max_iter': 300, 'clf__C': 10}



### ComplementNB

In [54]:
params_cnb = {'alpha': [0.01, 0.1, 0.5, 1.0, 10.0],
          'fit_prior': [True, False],
          'norm': [True, False],         
         }

t0 = time()

model_cnb = RandomizedSearchCV(ComplementNB(), 
                               param_distributions=params_cnb, 
                               n_jobs=-1, 
                               cv=3, 
                               n_iter=7, 
                               verbose=2, 
                               random_state=RANDOM_STATE, 
                               scoring='f1')

model_cnb.fit(tf_idf, target_train)

print(f"ComplementNB выполнена за {time() - t0:.3f}s")

print('F1 через RandomizedSearchCV: {:.3f}'.format(model_cnb.best_score_))
print('Лучшие параметры: {}\n'.format(model_cnb.best_params_))

Fitting 3 folds for each of 7 candidates, totalling 21 fits
ComplementNB выполнена за 4.783s
F1 через RandomizedSearchCV: 0.622
Лучшие параметры: {'norm': True, 'fit_prior': False, 'alpha': 0.1}



In [55]:
model_f1['F1']['ComplementNB'] = round(model_cnb.best_score_,4)

### RandomForestClassifier

In [56]:
params_rfc = {#'bootstrap': [True],
              'max_depth': [70, 90],
              #'max_features': ['auto', 'sqrt'],
              'min_samples_leaf': [4, 6],
              'min_samples_split': [5, 10],
              'n_estimators': [300, 400]
             }

In [57]:
t0 = time()

rfc_rscv = RandomizedSearchCV(RandomForestClassifier(), 
                              param_distributions=params_rfc, 
                              cv=3,  
                              n_iter=7,
                              scoring='f1', 
                              n_jobs=-1, 
                              random_state=RANDOM_STATE, 
                              verbose=2)

rfc_rscv.fit(tf_idf, target_train)

print(f"RandomForestClassifier выполнена за {time() - t0:.3f}s")

print(f'Лучшие параметры RandomForestClassifier: {rfc_rscv.best_params_}') 
print('Средняя оценка качества F1: %0.2f' % abs(rfc_rscv.best_score_))

Fitting 3 folds for each of 7 candidates, totalling 21 fits
RandomForestClassifier выполнена за 5026.244s
Лучшие параметры RandomForestClassifier: {'n_estimators': 400, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_depth': 90}
Средняя оценка качества F1: 0.14


In [58]:
model_f1['F1']['RandomForestClassifier'] = round(rfc_rscv.best_score_,4)

###  SGDClassifier

In [59]:
param_sgd = {
    'alpha': [1e-4, 1e-3, 1e-2, 1e-1, 1e0],
    'penalty': ['l1', 'l2'],
    'max_iter' : [500, 800, 1100],
    'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
    'learning_rate' : ['constant', 'optimal', 'invscaling', 'adaptive'],
    'eta0' : [1, 10, 100]
}

In [60]:
t0 = time()

model_sgd = RandomizedSearchCV(estimator = SGDClassifier(), 
                               param_distributions = param_sgd, 
                               n_iter = 7, 
                               cv = 3, 
                               verbose = 2, 
                               random_state=RANDOM_STATE, 
                               n_jobs = -1, 
                               scoring = 'f1')

model_sgd.fit(tf_idf, target_train)

print(f"SGDClassifier выполнена за {time() - t0:.3f}s")

f1_sgd = abs(model_sgd.best_score_)

print(f'Лучшие параметры SGDClassifier: {model_sgd.best_params_}') 
print('F1 SGDClassifier: %0.2f' % f1_sgd)

Fitting 3 folds for each of 7 candidates, totalling 21 fits
SGDClassifier выполнена за 21.368s
Лучшие параметры SGDClassifier: {'penalty': 'l2', 'max_iter': 800, 'loss': 'perceptron', 'learning_rate': 'optimal', 'eta0': 100, 'alpha': 0.001}
F1 SGDClassifier: 0.73


In [61]:
model_f1['F1']['SGDClassifier'] = round(f1_sgd,4)  

In [62]:
model_f1

,F1
LogisticRegression,0.7706
ComplementNB,0.622
RandomForestClassifier,0.1409
SGDClassifier,0.7282


Лучшая метрика F1 на учебной выборке получилась у логистической регрессии. 

In [74]:
print('F1 на тестовой выборке у LogisticRegression: %0.2f' % f1_score(model_lr.predict(features_test), target_test))

F1 на тестовой выборке у LogisticRegression: 0.79


## Выводы

1. Загружены и подготовлены данные.
2. Обучены разные модели.
3. Модель со значением метрики качества F1 >= 0.75: Логистическая регрессия.